# Session 3 - Random Forest Model Tuning

This is an introductory notebook to familiarize yourself with optimization techniques for Machine Learning (ML). For this work, we will use data from the Moderate Resolution Imaging Spectroradiometer (MODIS) instrument  and a Random Forest algorithm to perform water classification. This same workflow can be adapted to other applications and by using other algorithms.

Author: Caleb S. Spradlin, Jordan A. Caraballo-Vega
Release Date: 2023.04.08
Last Modified: 2023.04.08

## 1. Import Libraries

In this section we import the Python libraries to use during the development of this notebook. The default Python kernel from Google Colab does not include all fo the packages we need, thus we proceed to install them via pip.


In [ ]:
!wget https://raw.githubusercontent.com/NASAARSET/ARSET_ML_Fundamentals/main/src/folium_helper.py

In [ ]:
!pip install datasets optuna rasterio pyproj

In [ ]:
import os
import sys
import csv
import datetime
import glob
import joblib
import numpy as np
import datasets
import pandas as pd
from pathlib import Path   
import optuna

from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier as skRF
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, f1_score
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.inspection import permutation_importance
from huggingface_hub import snapshot_download

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

# Geospatial related imports
from osgeo import gdalconst
from osgeo import gdal
import folium
from folium import plugins
import folium_helper

from pprint import pprint

plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline

## 2. Define General Variables

In this section we define general variables to work with through this notebook. A description of each variable is listed below as a comment next to the variable definition.

In [ ]:
FIGURE_OUTPUT_DIR = 'output'
RASTER_OUTPUT_DIR = 'output'
MODEL_OUTPUT_DIR = 'models'

# url of the dataset we will be using, this is a link to the Hugging Face repository
# of this tutorial
DATASET_URL = 'nasa-cisto-data-science-group/modis-lake-powell-toy-dataset'

TILE = 'global'
MODEL = 'rf'
TEST_RATIO = 0.2
RANDOM_STATE = 42
LABEL_NAME = 'water'
DATA_TYPE = np.int16
colsToDrop = []#['x_offset', 'y_offset']
v_names = ['sur_refl_b01_1','sur_refl_b02_1','sur_refl_b03_1',
           'sur_refl_b04_1','sur_refl_b05_1','sur_refl_b06_1',
           'sur_refl_b07_1','ndvi','ndwi1','ndwi2']



Here we create an output directory to store any artifacts out of our models and visualizations.

In [ ]:
os.makedirs(MODEL_OUTPUT_DIR, exist_ok=True)
os.makedirs(FIGURE_OUTPUT_DIR, exist_ok=True)

## Data
Read in to cuDF or pandas Dataframe
Drop unnecessary columns
Clean data
Split into Xs and Ys
Train-test split

In [ ]:
%%time
train_dataset = pd.DataFrame(datasets.load_dataset(DATASET_URL, split='train'))
test_dataset = pd.DataFrame(datasets.load_dataset(DATASET_URL, split='test'))

In [ ]:
X_train, y_train = train_dataset.drop(['water'], axis=1), train_dataset['water']
X_test, y_test = test_dataset.drop(['water'], axis=1), test_dataset['water']
X_train.shape, X_test.shape

In [ ]:
_ = [print(column) for column in X_train.columns]


In [ ]:
X_train.describe().T

In [ ]:
def output_interesting_idx(df, column, threshold, greaterThan=True):
    dfToReturn = df[df[column] > threshold] if \
        greaterThan else df[df[column] < threshold]
    return dfToReturn

In [ ]:
output_interesting_idx(X_train, 'ndvi', 10000)


In [ ]:
output_interesting_idx(X_train, 'ndwi1', 10000)


In [ ]:
output_interesting_idx(X_train, 'ndwi2', 10000)


## Model definition and training
Set TREES_ONLY to True if you only want to tune number of estimators, else max_depth, max_samples_split, min_samples_leaf will be tuned as well.

In [ ]:
TREES_ONLY = True

In [ ]:
def objective(trial):
    list_trees = [75, 100, 125, 150, 175, 200, 250, 300, 400, 500]
    max_depth = [80, 90, 100, 110]
    min_samples_leaf = [1, 2, 3, 4, 5]
    min_samples_split = [2, 4, 8, 10]
    bootstrap = [True, False]
    
    if TREES_ONLY:
        param = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees), 
                   'criterion':'gini', 
                   'max_depth':None, 
                   'min_samples_split':2, 
                   'min_samples_leaf':1, 
                   'min_weight_fraction_leaf':0.0, 
                   'max_features':'auto', 
                   'max_leaf_nodes':None, 
                   'min_impurity_decrease':0.0, 
                   'bootstrap':True, 
                   'oob_score':False, 
                   'n_jobs':-1, 
                   'random_state':42, 
                   'verbose':0, 
                   'warm_start':True, 
                   'class_weight':None, 
                   'ccp_alpha':0.0, 
                   'max_samples':None
                      }
    else:
        param = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees), 
                       'max_depth':trial.suggest_categorical('max_depth', max_depth), 
                       'min_samples_split':trial.suggest_categorical('min_samples_split', min_samples_split), 
                       'min_samples_leaf':trial.suggest_categorical('min_samples_leaf', min_samples_leaf), 
                       'bootstrap': True,
                       'criterion':'gini', 
                       'min_weight_fraction_leaf':0.0, 
                       'max_features':'auto', 
                       'max_leaf_nodes':None, 
                       'min_impurity_decrease':0.0, 
                       'oob_score':False, 
                       'n_jobs':-1, 
                       'random_state':42, 
                       'verbose':0, 
                       'warm_start':False, 
                       'class_weight':None, 
                       'ccp_alpha':0.0, 
                       'max_samples':None
                      }
    
    rf = skRF(**param)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)
    cm = confusion_matrix(y_test, preds)
    fp = cm[1][0]
    precision = precision_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    del rf, preds, cm
    return precision, f1, fp

metric = {'precision': 0, 'f1': 1, 'false_positive': 2}

## Start hyperparameter tuning trials.
Ex output: Trial 0 finished with values: [0.9527224325581133, 0.9667626330841518, 9684.0] and parameters:

The metrics in order are: [precision, f1, false positives]

In [ ]:
optuna.logging.set_verbosity(optuna.logging.INFO)
study = optuna.create_study(study_name='rf_tuning', directions=['maximize','maximize','minimize'])
study.optimize(objective, n_trials=25, timeout=20*60)

Get metrics from the overall hyperparameter tuning


In [ ]:
METRIC_TO_USE = 'f1'# out of 'precision', 'f1', 'false_positive'

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Using {} idx: {}".format(METRIC_TO_USE, metric[METRIC_TO_USE]))
trials = study.best_trials
if METRIC_TO_USE == 'false_positive' or METRIC_TO_USE == 'false_negative':
    trial_score = min([trial.values[metric[METRIC_TO_USE]] for trial in trials])
else:
    trial_score = max([trial.values[metric[METRIC_TO_USE]] for trial in trials])
best_trial_params = [trial.params for trial in trials if trial.values[metric[METRIC_TO_USE]] == trial_score][0]
best_trial = [trial for trial in trials if trial.values[metric[METRIC_TO_USE]] == trial_score][0]
print(best_trial_params)
print(trial_score)

trial_scores = [trial.values for trial in trials]
trial_params = [trial.params for trial in trials]

for score, param in zip(trial_scores, trial_params):
    print(score)
    for k, v in param.items():
        print("     {}: {}".format(k, v))

study_df = study.trials_dataframe()
study_df.to_csv("hyperopt_tuning_trial_{}_random_forest.csv".format(
    datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')))

Visualize the optimization history


In [ ]:
optuna.visualization.matplotlib.plot_optimization_history(study, target=lambda t: t.values[metric[METRIC_TO_USE]])

Visualize hyperparameter importances


In [ ]:
optuna.visualization.matplotlib.plot_param_importances(study, target=lambda t: t.values[metric[METRIC_TO_USE]])

Visualize the loss functions compares to number of estimators during tuning


In [ ]:
udf = study_df.drop_duplicates(['params_n_estimators']).sort_values(by='params_n_estimators')
udf_by_score = udf[udf.values_0 == udf.values_0.max()]
udf

In [ ]:
if TREES_ONLY:
    udf = study_df.drop_duplicates(['params_n_estimators']).sort_values(by='params_n_estimators')
    udf_by_score = udf[udf.values_0 == udf.values_0.max()]
    fig, ax = plt.subplots(3, figsize=(12,15))
    ax[0].plot(udf['params_n_estimators'], udf['values_0'], label='Precision per n estimators')
    ax[0].set_xlabel('Number of Estimators')
    ax[0].set_ylabel('Precision Score')
    ax[1].plot(udf['params_n_estimators'], udf['values_1'], label='F1 score per n estimators')
    ax[1].set_xlabel('Number of Estimators')
    ax[1].set_ylabel('F1 Score')
    ax[2].plot(udf['params_n_estimators'], udf['values_2'], label='False positives per n estimators')
    ax[2].set_xlabel('Number of estimators')
    ax[2].set_ylabel('False Positives')

Training the model with the best hyperparamers


In [ ]:
hyperparameters = best_trial_params
hyperparameters['n_jobs'] = -1
hyperparameters['warm_start'] = True
print('Using these params:')
pprint(hyperparameters)
classifier = skRF(**hyperparameters)

In [ ]:
kf = KFold(n_splits=5)
kf.get_n_splits(X_train)

In [ ]:
bestModel = None
bestModelScore = 0
scores = []
for trainIdx, testIdx in kf.split(X_train):
    print("Train {}, Test {}".format(trainIdx, testIdx))
    X_train_valid, X_test_valid = X_train.iloc[trainIdx], X_train.iloc[testIdx]
    y_train_valid, y_test_valid = y_train.iloc[trainIdx], y_train.iloc[testIdx]
    print('Fitting model')
    classifier.fit(X_train_valid, y_train_valid)
    print('Getting score')
    score = classifier.score(X_test_valid, y_test_valid)
    if score>=bestModelScore:
        bestModelScore = score
        print('Training accuracy score: {}'.format(score))
        bestModel = classifier
    print('Predicting for test set')
    test_predictions = classifier.predict(X_test_valid)
    print(classification_report(y_test_valid, test_predictions))
    print('Score: {}'.format(score))
    scores.append(score)
    del test_predictions, score

In [ ]:
scoreAvg = np.asarray(scores).mean()
print('Average accuracy score: {}'.format(scoreAvg))
print('Best accuracy score: {}'.format(bestModelScore))

In [ ]:
classifier = bestModel

In [ ]:
score = classifier.score(X_test, y_test)
score = round(score, 3)
score

In [ ]:
train_predictions = classifier.predict(X_train)
test_predictions = classifier.predict(X_test)
prediction_probs = classifier.predict_proba(X_test)

In [ ]:
test_predictions = test_predictions.astype(np.int16)
y_test_int = y_test.astype(np.int16)

In [ ]:
print('Train Performance')
print('-------------------------------------------------------')
print(classification_report(y_train, train_predictions))
print('Test Performance')
print('-------------------------------------------------------')
print(classification_report(y_test, test_predictions))
cm = confusion_matrix(y_test_int, test_predictions)
recall = (cm[0][0] / (cm[0][0] + cm[0][1]))
print('Test Recall')
print('-------------------------------------------------------')
print(recall)
print('Confusion Matrix')
print('-------------------------------------------------------')
print(cm)

In [ ]:
%%time
permutation_importance_results = permutation_importance(classifier,
                                                        X=X_test,
                                                        y=y_test,
                                                        n_repeats=10,
                                                        random_state=42)

In [ ]:
sorted_idx = permutation_importance_results.importances_mean.argsort()
plt.figure(figsize=(8, 8))
plt.barh(X_test.columns[sorted_idx], permutation_importance_results.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

In [ ]:
del X_train, X_test, y_train, y_test, test_predictions, train_predictions, prediction_probs, y_test_int


In [ ]:
model_save_path = './water_classifier_rf_cpu.sav'
print('Saving model to: {}'.format(model_save_path))
print(classifier)
joblib.dump(classifier, model_save_path, compress=3)

In [ ]:
powell_dataset = snapshot_download(repo_id=DATASET_URL, allow_patterns="*.tif", repo_type='dataset')
fileList = sorted([file for file in glob.glob(os.path.join(powell_dataset, 'IL.*.Powell.*.tif')) if 'sur_refl' in file])
fileList

In [ ]:
def readRastersToArray(fileList):
    rasterProjection = None
    newshp = (1300*1300, 10)
    img = np.empty(newshp, dtype=np.int16)
    for i, fileName in enumerate(fileList):
        ds = gdal.Open(fileName)
        img[:, i] = ds.GetRasterBand(1).ReadAsArray().astype(np.int16).ravel()
        if i == 0:
            rasterProjection = ds.GetProjection()
        ds = None
    img[:, len(fileList)] = ((img[:, 1] - img[:, 0]) / (img[:, 1] + img[:, 0])) * 10000
    img[:, len(fileList)+1] = ((img[:, 1] - img[:, 5]) / (img[:, 1] + img[:, 5])) * 10000
    img[:, len(fileList)+2] = ((img[:, 1] - img[:, 6]) / (img[:, 1] + img[:, 6])) * 10000
    return img, rasterProjection

In [ ]:
%%time
im, rasterProjection = readRastersToArray(fileList)
print('Raster as ndarray')
print(im)
print('{} MB size'.format((im.size * im.itemsize) / 1000000))

In [ ]:

def predictRaster(img_chunk, colsToDrop=None):
    """
    Function given a raster in the form of a nxn matrix, will
    convert the matrix to a GPU-bound data frame then perform 
    predictions given the loaded model.
    
    Return the prediction matrix, the prediction probabilities
    for each and the dataframe converted to host.
    """
    print('Converting host array to CPU-based dataframe')
    df = pd.DataFrame(img_chunk, columns=v_names, dtype=np.float32)
    df = df.drop(columns=colsToDrop)
    print('Making predictions from raster')
    predictions = classifier.predict(df)
    predictionsProbs = classifier.predict_proba(df)
    return predictions, predictionsProbs, df

In [ ]:
%%time
predictedRaster, predictedProbaRaster, df = predictRaster(im, colsToDrop)

In [ ]:
shp = (1300, 1300)
left = list()
right = list()
for i, subarr in enumerate(predictedProbaRaster):
    left.append(subarr[0])
    right.append(subarr[1])
leftArr = np.asarray(left)
rightArr = np.asarray(right)
probaLand = leftArr.reshape(shp)
probaWater = rightArr.reshape(shp)

In [ ]:
shp = (1300, 1300)
matrix = np.asarray(predictedRaster)
reshp = matrix.reshape(shp)
reshp.shape

In [ ]:
qa = [file for file in glob.glob(os.path.join(powell_dataset, '*.tif')) if 'qa' in file][0]
ds = gdal.Open(qa)
qaMask = ds.GetRasterBand(1).ReadAsArray()
output = np.where(qaMask == 0, reshp, -9999)
qaMask

In [ ]:
countNoData = np.count_nonzero(output == -9999)
countLand = np.count_nonzero(output == 0)
countWater = np.count_nonzero(output == 1)
print('Predicted\n No-data occuraces: {}\n Land occurances: {}\n Water occurances: {}'.format(countNoData, countLand, countWater))

In [ ]:
geoTransform = (-9961223.407, 231.65635, 0.0, 4285642.633667, 0.0, -231.65635)


In [ ]:
predictedPath = os.path.join(RASTER_OUTPUT_DIR, 'PowellPredictedWaterMask.tif')

driver = gdal.GetDriverByName('GTiff')
outDs = driver.Create(predictedPath, 1300, 1300, 1, gdal.GDT_Int16, options=['COMPRESS=LZW'])
outDs.SetGeoTransform(geoTransform)
outDs.SetProjection(rasterProjection)
outBand = outDs.GetRasterBand(1)
outBand.WriteArray(output)
outBand.SetNoDataValue(-9999)
outDs.FlushCache()
outDs = None
outBand = None
driver = None

In [ ]:

mask_3857 = folium_helper.reproject_to_3857(predictedPath)
mask_d = folium_helper.get_bounds(mask_3857)
mask_b1 = folium_helper.open_and_get_band(mask_3857, 1)
folium_helper.cleanup(mask_3857)
mask_b1 = np.where(mask_b1 == -9999, 0, mask_b1)
zeros = np.zeros_like(mask_b1)
mask_rgb = np.dstack((mask_b1, zeros, zeros))

In [ ]:
m = folium.Map(location=[mask_d['center'][1], mask_d['center'][0]],
                   tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', zoom_start = 6, attr='Google')

In [ ]:
m.add_child(folium_helper.get_overlay(mask_rgb, mask_d, 'Water classification RF predicted mask', opacity=0.6))
m.add_child(plugins.MousePosition())
m.add_child(folium.LayerControl())